# Lab Results Analysis - Data Collection Data

**This notebook analyzes lab results data from your Workbench data collection and generates distribution reports.**

## 📊 What You'll See

After running all cells, you'll get:
- **Data Overview**: Summary statistics and data structure
- **Distribution Reports**: For Patient ID, Lab Type, Lab Value, and Lab Date
- **Visualizations**: Charts, graphs, and statistical analyses
- **Summary Statistics**: Comprehensive data characteristics

## 🚀 Quick Start

Just click **"Run All"** to analyze your data from the data collection bucket!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
from pathlib import Path
warnings.filterwarnings('ignore')

# Optional: Import google.cloud.storage (installed automatically if needed)
try:
    from google.cloud import storage
    GCS_AVAILABLE = True
except ImportError:
    GCS_AVAILABLE = False
    print("ℹ️  Installing google-cloud-storage...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-cloud-storage"])
    from google.cloud import storage
    GCS_AVAILABLE = True
    print("✅ google-cloud-storage installed successfully!")

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ============================================================================
# CONFIGURATION: Data Collection Bucket and File
# ============================================================================
GCS_BUCKET = "My_GCS_Experimentation_Bucker"  # Your data collection bucket
FILE_NAME = "MUP_DPR_RY25_P04_V10_DY23_Geo.csv"   # Your data file
FILE_FORMAT = "csv"  # File format

print("="*70)
print("📊 Configuration")
print("="*70)
print(f"Bucket: {GCS_BUCKET}")
print(f"File: {FILE_NAME}")
print(f"Format: {FILE_FORMAT}")
print(f"GCS Path: gs://{GCS_BUCKET}/{FILE_NAME}")
print("="*70)


In [ ]:
## 2. Load Data from GCS Bucket


In [ ]:
def load_data_from_gcs(bucket_name, file_name, file_format="csv"):
    """Load data from GCS bucket using Google Cloud Storage client."""
    try:
        # Initialize GCS client
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        
        print(f"📥 Reading file from GCS: gs://{bucket_name}/{file_name}")
        
        # Download to temporary file
        temp_file = f"/tmp/{os.path.basename(file_name)}"
        blob.download_to_filename(temp_file)
        print(f"✅ File downloaded to: {temp_file}")
        
        # Read based on file format
        if file_format.lower() == "csv":
            df = pd.read_csv(temp_file)
        elif file_format.lower() == "parquet":
            df = pd.read_parquet(temp_file)
        elif file_format.lower() == "json":
            df = pd.read_json(temp_file)
        elif file_format.lower() == "excel":
            df = pd.read_excel(temp_file)
        else:
            raise ValueError(f"Unsupported file format: {file_format}")
        
        # Clean up temp file
        os.remove(temp_file)
        print(f"✅ Data loaded successfully: {len(df)} rows, {len(df.columns)} columns")
        return df
        
    except Exception as e:
        print(f"❌ Error loading from GCS: {e}")
        raise

# Load data from GCS
print("\n" + "="*70)
print("Loading data from data collection...")
print("="*70)
bucket_name = GCS_BUCKET.replace("gs://", "").strip()
df = load_data_from_gcs(bucket_name, FILE_NAME, FILE_FORMAT)

# Ensure required columns exist (case-insensitive matching)
required_columns = ['Patient ID', 'Lab Type', 'Lab Value', 'Lab Date']
df_columns_lower = {col.lower(): col for col in df.columns}

# Map to standard column names
column_mapping = {}
for req_col in required_columns:
    req_lower = req_col.lower()
    if req_lower in df_columns_lower:
        column_mapping[df_columns_lower[req_lower]] = req_col
    else:
        print(f"⚠️  WARNING: Column '{req_col}' not found in data. Available columns: {list(df.columns)}")

if column_mapping:
    df = df.rename(columns=column_mapping)

# Ensure Lab Date is datetime
if 'Lab Date' in df.columns:
    df['Lab Date'] = pd.to_datetime(df['Lab Date'], errors='coerce')

print(f"\n✅ Dataset ready: {len(df)} records")
print(f"📋 Columns: {list(df.columns)}")
print(f"\n📊 First few records:")
df.head(10)


## 2. Data Overview


In [ ]:
print("Dataset Info:")
print(f"Total Records: {len(df)}")
print(f"Total Patients: {df['Patient ID'].nunique()}")
print(f"Total Lab Types: {df['Lab Type'].nunique()}")
print(f"\nDate Range: {df['Lab Date'].min().date()} to {df['Lab Date'].max().date()}")
print(f"\nData Types:")
print(df.dtypes)
print(f"\nBasic Statistics:")
df.describe()


## 3. Distribution Report: Patient ID


In [ ]:
# Count of lab results per patient
patient_counts = df['Patient ID'].value_counts().sort_values(ascending=False)

print("=== Patient ID Distribution Report ===")
print(f"\nTotal unique patients: {df['Patient ID'].nunique()}")
print(f"\nTop 10 patients by number of lab results:")
print(patient_counts.head(10))
print(f"\nStatistics:")
print(f"  Mean tests per patient: {patient_counts.mean():.2f}")
print(f"  Median tests per patient: {patient_counts.median():.2f}")
print(f"  Min tests per patient: {patient_counts.min()}")
print(f"  Max tests per patient: {patient_counts.max()}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of tests per patient
axes[0].hist(patient_counts.values, bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_xlabel('Number of Lab Tests', fontsize=12)
axes[0].set_ylabel('Number of Patients', fontsize=12)
axes[0].set_title('Distribution of Lab Tests per Patient', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Top 15 patients bar chart
top_patients = patient_counts.head(15)
axes[1].barh(range(len(top_patients)), top_patients.values, color='coral')
axes[1].set_yticks(range(len(top_patients)))
axes[1].set_yticklabels(top_patients.index, fontsize=9)
axes[1].set_xlabel('Number of Lab Tests', fontsize=12)
axes[1].set_title('Top 15 Patients by Lab Test Count', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


## 4. Distribution Report: Lab Type


In [ ]:
# Count of each lab type
lab_type_counts = df['Lab Type'].value_counts()

print("=== Lab Type Distribution Report ===")
print(f"\nTotal unique lab types: {df['Lab Type'].nunique()}")
print(f"\nLab type frequency:")
for lab_type, count in lab_type_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {lab_type}: {count} ({percentage:.1f}%)")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
axes[0].barh(range(len(lab_type_counts)), lab_type_counts.values, color='lightgreen')
axes[0].set_yticks(range(len(lab_type_counts)))
axes[0].set_yticklabels(lab_type_counts.index, fontsize=10)
axes[0].set_xlabel('Number of Tests', fontsize=12)
axes[0].set_title('Lab Type Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Pie chart
axes[1].pie(lab_type_counts.values, labels=lab_type_counts.index, autopct='%1.1f%%', 
            startangle=90, textprops={'fontsize': 9})
axes[1].set_title('Lab Type Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


## 5. Distribution Report: Lab Value


In [ ]:
print("=== Lab Value Distribution Report ===")
print(f"\nBasic Statistics:")
print(df['Lab Value'].describe())
print(f"\nSkewness: {df['Lab Value'].skew():.3f}")
print(f"Kurtosis: {df['Lab Value'].kurtosis():.3f}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(df['Lab Value'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Lab Value', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Lab Value Distribution (Histogram)', fontsize=14, fontweight='bold')
axes[0, 0].axvline(df['Lab Value'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Lab Value"].mean():.2f}')
axes[0, 0].axvline(df['Lab Value'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["Lab Value"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot
axes[0, 1].boxplot(df['Lab Value'], vert=True, patch_artist=True, 
                   boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0, 1].set_ylabel('Lab Value', fontsize=12)
axes[0, 1].set_title('Lab Value Distribution (Box Plot)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Density plot
df['Lab Value'].plot.density(ax=axes[1, 0], color='purple', linewidth=2)
axes[1, 0].set_xlabel('Lab Value', fontsize=12)
axes[1, 0].set_ylabel('Density', fontsize=12)
axes[1, 0].set_title('Lab Value Distribution (Density Plot)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Lab values by lab type
df.boxplot(column='Lab Value', by='Lab Type', ax=axes[1, 1], rot=45)
axes[1, 1].set_xlabel('Lab Type', fontsize=10)
axes[1, 1].set_ylabel('Lab Value', fontsize=12)
axes[1, 1].set_title('Lab Value Distribution by Lab Type', fontsize=14, fontweight='bold')
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45, ha='right')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 6. Distribution Report: Lab Date


In [ ]:
# Extract date components
df['Year'] = df['Lab Date'].dt.year
df['Month'] = df['Lab Date'].dt.month
df['DayOfWeek'] = df['Lab Date'].dt.day_name()

print("=== Lab Date Distribution Report ===")
print(f"\nDate Range: {df['Lab Date'].min().date()} to {df['Lab Date'].max().date()}")
print(f"\nTotal days covered: {(df['Lab Date'].max() - df['Lab Date'].min()).days} days")
print(f"\nTests by Year:")
print(df['Year'].value_counts().sort_index())
print(f"\nTests by Month:")
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_counts = df['Month'].value_counts().sort_index()
for month, count in month_counts.items():
    print(f"  {month_names[month-1]}: {count}")
print(f"\nTests by Day of Week:")
print(df['DayOfWeek'].value_counts())

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Timeline of tests
daily_counts = df.groupby(df['Lab Date'].dt.date).size()
axes[0, 0].plot(daily_counts.index, daily_counts.values, marker='o', markersize=3, linewidth=1, color='darkblue')
axes[0, 0].set_xlabel('Date', fontsize=12)
axes[0, 0].set_ylabel('Number of Tests', fontsize=12)
axes[0, 0].set_title('Lab Tests Over Time', fontsize=14, fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Tests by month
month_counts = df['Month'].value_counts().sort_index()
axes[0, 1].bar(range(1, 13), [month_counts.get(i, 0) for i in range(1, 13)], color='orange', alpha=0.7)
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(month_names, rotation=45, ha='right')
axes[0, 1].set_xlabel('Month', fontsize=12)
axes[0, 1].set_ylabel('Number of Tests', fontsize=12)
axes[0, 1].set_title('Lab Tests by Month', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Tests by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = df['DayOfWeek'].value_counts().reindex(day_order, fill_value=0)
axes[1, 0].bar(range(len(day_order)), day_counts.values, color='teal', alpha=0.7)
axes[1, 0].set_xticks(range(len(day_order)))
axes[1, 0].set_xticklabels(day_order, rotation=45, ha='right')
axes[1, 0].set_xlabel('Day of Week', fontsize=12)
axes[1, 0].set_ylabel('Number of Tests', fontsize=12)
axes[1, 0].set_title('Lab Tests by Day of Week', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Histogram of dates
axes[1, 1].hist(df['Lab Date'], bins=30, edgecolor='black', alpha=0.7, color='crimson')
axes[1, 1].set_xlabel('Date', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Lab Date Distribution (Histogram)', fontsize=14, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7. Summary Statistics


In [ ]:
print("="*70)
print(" " * 15 + "📊 COMPREHENSIVE SUMMARY REPORT 📊")
print("="*70)
print(f"\n{'='*70}")
print(f"📋 DATASET OVERVIEW")
print(f"{'='*70}")
print(f"  Total Records: {len(df):,}")
print(f"  Date Range: {df['Lab Date'].min().date()} to {df['Lab Date'].max().date()}")
print(f"  Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n{'='*70}")
print(f"👥 PATIENT ID ANALYSIS")
print(f"{'='*70}")
print(f"  Total Unique Patients: {df['Patient ID'].nunique():,}")
print(f"  Average Tests per Patient: {df.groupby('Patient ID').size().mean():.2f}")
print(f"  Median Tests per Patient: {df.groupby('Patient ID').size().median():.2f}")
print(f"  Patients with Most Tests: {df.groupby('Patient ID').size().max()} tests")

print(f"\n{'='*70}")
print(f"🧪 LAB TYPE ANALYSIS")
print(f"{'='*70}")
print(f"  Total Unique Lab Types: {df['Lab Type'].nunique()}")
most_common = df['Lab Type'].mode()[0]
most_common_count = df['Lab Type'].value_counts().max()
most_common_pct = (most_common_count / len(df)) * 100
print(f"  Most Common Lab Type: {most_common} ({most_common_count} tests, {most_common_pct:.1f}%)")
print(f"  Lab Type Distribution:")
for lab_type, count in df['Lab Type'].value_counts().head(5).items():
    pct = (count / len(df)) * 100
    print(f"    - {lab_type}: {count} ({pct:.1f}%)")

print(f"\n{'='*70}")
print(f"📈 LAB VALUE STATISTICS")
print(f"{'='*70}")
print(f"  Mean: {df['Lab Value'].mean():.2f}")
print(f"  Median: {df['Lab Value'].median():.2f}")
print(f"  Standard Deviation: {df['Lab Value'].std():.2f}")
print(f"  Minimum: {df['Lab Value'].min():.2f}")
print(f"  Maximum: {df['Lab Value'].max():.2f}")
print(f"  Range: {df['Lab Value'].max() - df['Lab Value'].min():.2f}")
print(f"  Skewness: {df['Lab Value'].skew():.3f}")

print(f"\n{'='*70}")
print(f"📅 TEMPORAL ANALYSIS")
print(f"{'='*70}")
date_range_days = (df['Lab Date'].max() - df['Lab Date'].min()).days
print(f"  Date Range: {date_range_days} days")
print(f"  Average Tests per Day: {len(df) / (date_range_days + 1):.2f}")
print(f"  Total Tests: {len(df):,}")

# Find busiest day
daily_counts = df.groupby(df['Lab Date'].dt.date).size()
busiest_day = daily_counts.idxmax()
busiest_count = daily_counts.max()
print(f"  Busiest Day: {busiest_day} ({busiest_count} tests)")

print(f"\n{'='*70}")
print("✅ Analysis Complete! All distribution reports and visualizations are shown above.")
print(f"{'='*70}")
